In [ ]:
# github test

In [24]:
import tweepy
import re
import pandas as pd
import numpy as np
from textblob import TextBlob
import json

In [2]:
# # Anran's access
# each access, as sandbox account, has limit of 50 requests and 5k tweets for full archive search

# consumer_key = "yJI7VztRV1IFd52g5Qm6CoQ4k"   
# consumer_secret = "yN2z5rlWziN9Wdi9lB9XjLvogl5ItqXoBlcvck8S2N7mj3FtfF"   
# access_token = "909527697646395393-HUEr0Aws7DCnn3VWGrWnMakmPY0SlAI"
# access_token_secret = "4ntwDjAKihF9ZKELxMaxRc4GqMd1A9yAamlbmFF7kjCtE"


In [3]:
# Joe's access

consumer_key = "5IeUaLOGKCfAB1BPvFS80U5to"   
consumer_secret = "CwkGUnf3pACH2fZNT7JJpRqWZcsa2GrWkqXqA7TPNX5F4FgCes"   
access_token = "3008283568-mtwZBWaDf69tS83YPQRV3XqTnDUtwGipcRpYZNw"
access_token_secret = "S09bX72JGQDdAyhP7j6u3cUKrInNXAOzIftoJs27EmkuS"


In [4]:
# Tweepy settings
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)   
auth.set_access_token(access_token, access_token_secret)   
api = tweepy.API(auth)   


In [45]:
def search_tweet(environmentname, fromdate, todate, keywords, searchsize, savedfilename): 
# e.g. envronment name='Development', query='vegan', fromDate='201101012315', toDate='201102012315', searchsize =100
# for Anran's access, env name is 'Development', for Joe, 'research'
# search size should be at least 100, and be like 100, 200, 300 etc. 

    idlist=[]
    tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name=environmentname, query=keywords, fromDate=fromdate, toDate=todate,).items(searchsize)
    for tweet in tweets: 
        idlist.append(tweet.id)
    
    
    columns = set()
#     allowed_types = []
    tweets_data = []
    

    while len(idlist)>=100:
        idlistcurr=idlist[:100]
        status2=api.statuses_lookup(idlistcurr,tweet_mode="extended")


        for status in status2:
            status_dict = dict(vars(status))
            keys = status_dict.keys()
            single_tweet_data = {"user": status.user.screen_name}
            for k in keys:
                try:
                    v_type = type(status_dict[k])
                except: # make sure no none error
                    v_type = None
                if v_type != None: # make sure no none error
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
            tweets_data.append(single_tweet_data)
        idlist=idlist[100:]


    header_cols = list(columns)
    header_cols.append("user")

    df = pd.DataFrame(tweets_data, columns=header_cols)
    df.to_csv(savedfilename)
    
    
    return df


In [46]:
res1=search_tweet('research', '202104012315', '202104202315', 'vaccine uk', 1200, '2021Apr.csv')

In [49]:
res1.shape

(1200, 36)

In [17]:
res1.full_text[39]

'!!!sec #2524 of #health care #reform #bill includesprogram"No #Child Left Unimmunized Against #Influenza." http://t.co/w9dnY1h via @mercola'

In [13]:
# 



def search_tweet(fromdate, todate, keywords, searchsize): # e.g. query='vegan', fromDate='201101012315', toDate='201102012315',).items(100)
    idlist=[]
    tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query=keywords, fromDate=fromdate, toDate=todate,).items(searchsize)
    for tweet in tweets: 
        idlist.append(tweet.id)
    
    
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    created_time=[]

    while len(idlist)>=100:
        idlistcurr=idlist[:100]
        status2=api.statuses_lookup(idlistcurr,tweet_mode="extended")


        for status in status2:
            status_dict = dict(vars(status))
            keys = status_dict.keys()
            single_tweet_data = {"user": status.user.screen_name}
            for k in keys:
                try:
                    v_type = type(status_dict[k])
                except: # make sure no none error
                    v_type = None
                if v_type != None: # make sure no none error
                    if v_type in allowed_types:
                        single_tweet_data[k] = status_dict[k]
                        columns.add(k)
            tweets_data.append(single_tweet_data)
            created_time.append(status.created_at)
        idlist=idlist[100:]


    header_cols = list(columns)
    header_cols.append("user")

    df = pd.DataFrame(tweets_data, columns=header_cols)
    df['created_time']=created_time
    return df


In [43]:
# df.to_csv('sample 600 result.csv')

In [41]:
df.columns

Index(['in_reply_to_user_id_str', 'full_text', 'id', 'in_reply_to_screen_name',
       'in_reply_to_user_id', 'source_url', 'in_reply_to_status_id_str',
       'favorite_count', 'lang', 'source', 'retweet_count', 'id_str',
       'in_reply_to_status_id', 'user', 'created_time'],
      dtype='object')

In [36]:
# search 20 item and store them in df
# 'vegan', English(no), 2011 Jan-Feb
# handling retweets, show full text

# for Sandbox user, limits are 50 requests and 5k tweets per month. 
# there's no limit for the each search though.the last test for the searchsize is 600 and it works. 

idlist=[]
tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='201101012315', toDate='201102012315',).items(100)
for tweet in tweets: 
    idlist.append(tweet.id)

# if searchsize <=100:     
# status2=api.statuses_lookup(idlist,tweet_mode="extended")
# for status in status2:
#     try:
#         print(status.retweeted_status.full_text)
#     except AttributeError:  # Not a Retweet
#         print(status.full_text)

# if searchsize >100:

columns = set()
allowed_types = [str, int]
tweets_data = []
created_time=[]
    
while len(idlist)>=100:
    idlistcurr=idlist[:100]
    status2=api.statuses_lookup(idlistcurr,tweet_mode="extended")
    

    for status in status2:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"user": status.user.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except: # make sure no none error
                v_type = None
            if v_type != None: # make sure no none error
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)
        created_time.append(status.created_at)
    idlist=idlist[100:]

        
header_cols = list(columns)
header_cols.append("user")
# header_cols.append('created_time')

df = pd.DataFrame(tweets_data, columns=header_cols)
df['created_time']=created_time


TweepError: {'message': 'Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.', 'sent': '2021-04-26T10:52:53+00:00', 'transactionId': '00a174f100a23dc7'}

In [29]:
df = pd.DataFrame(tweets_data, columns=header_cols)
df['created_time']=created_time
df.head()


,in_reply_to_user_id_str,full_text,id,in_reply_to_screen_name,in_reply_to_user_id,source_url,in_reply_to_status_id_str,favorite_count,lang,source,retweet_count,id_str,in_reply_to_status_id,user,created_time
0,14205819,Are you a #vegan mom? Have some advice for nu...,32576188593475585,dopemamaJ,14205819.0,http://twitter.com,32558219918311424,0,en,Twitter Web Client,0,32576188593475585,3.255822e+16,vegansalt,2011-02-01 23:09:03
1,19468868,@Shep133 I had a vegan burger the other day wi...,32576131479633920,Shep133,19468868.0,http://twitter.com/download/android,32572192180936704,0,en,Twitter for Android,0,32576131479633920,3.257219e+16,leja99,2011-02-01 23:08:49
2,NaN,Overall GRATEFUL @TheOprahShow did a vegan sho...,32577369881452544,NaN,NaN,http://www.ubertwitter.com/bb/download.php,NaN,0,en,ÜberSocialOrig,4,32577369881452544,NaN,simonereyes,2011-02-01 23:13:44
3,114023803,@DreddedNupe @sableverity vegan mac n cheese i...,32577076951261184,RalphInCharge,114023803.0,http://www.echofon.com/,32575282254708737,0,en,Echofon,0,32577076951261184,3.257528e+16,SusannaDW,2011-02-01 23:12:35
4,NaN,RT @Chichi_C: Whyyyy?? **RT @naijafabulosity: ...,32576332147720192,NaN,NaN,http://www.ubertwitter.com/bb/download.php,NaN,0,en,ÜberSocialOrig,0,32576332147720192,NaN,Zeetherapy,2011-02-01 23:09:37


In [19]:
df.loc[4].full_text

"RT @Chichi_C: Whyyyy?? **RT @naijafabulosity: Making my vegan shopping list. &gt;Lol I cld say I'm trying to be healthier but to lose 10 pounds"

In [43]:
# store all features
# ref: from 26min, https://www.youtube.com/watch?v=dvAurfBB6Jk&t=2637s&ab_channel=CodingEntrepreneurs 
# need further editing. what's difference between author and user?


search=api.search(q='vegan', lang='en', count=5,tweet_mode='extended')

columns = set()
allowed_types = [str, int]
tweets_data = []
for status in search:
    status_dict = dict(vars(status))
    keys = status_dict.keys()
    single_tweet_data = {"user": status.user.screen_name}
    for k in keys:
        try:
            v_type = type(status_dict[k])
        except: # make sure no none error
            v_type = None
        if v_type != None: # make sure no none error
            if v_type in allowed_types:
                single_tweet_data[k] = status_dict[k]
                columns.add(k)
    tweets_data.append(single_tweet_data)

        
header_cols = list(columns)
header_cols.append("user")

df = pd.DataFrame(tweets_data, columns=header_cols)
df

500

In [16]:
# try full archive

tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='201101012315', toDate='201201012315').items(20)
for tweet in tweets:     
    print(tweet.text, tweet.created_at) 

$1 to the Ethiopian Homeless Feeding Program (vegan) for each e-mail sign-up in January (up to $3,000) --&gt; DAY 1... http://t.co/ICduihnY 2012-01-01 23:14:57
@enterblisstonia oooh. tofu seitan scramble from vegan yum yum (make your own seitan if you have a food processor)... 2012-01-01 23:14:50
“@lizabitty: Making vegan "beef" stew. Good way to start the year. Yummy smell throughout the house!” 2012-01-01 23:14:46
FGV flashback: vegan steak potato crisps/chips http://t.co/Y0mcyuqw 2012-01-01 23:14:36
Best raw, vegan, and organic foods http://marty_642.tripod.com/bestrawveganandorganicfoods.html 2012-01-01 23:14:33
Vegan for reviews http://t.co/8K9xSLeB 2012-01-01 23:14:24
Making vegan "beef" stew. Good way to start the year. Yummy smell throughout the house! 2012-01-01 23:14:14
RT @OneGreenPlanet: Happy New Year One Green Planeteers! Looking forward to spreading more good ideas for people, animals and the planet ... 2012-01-01 23:14:12
having some awesome @Engine2Diet #vegan gnocchi 

In [27]:
# try get tweet id from full archive, and use id to get full text 

idlist=[]
tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='202001012315', toDate='202001022315').items(20)
for tweet in tweets:     
    idlist.append(tweet.id)
    


In [28]:
idlist

[1212874962144509953,
 1212874956620664832,
 1212874954972246017,
 1212874946801815552,
 1212874939247845377,
 1212874921723998208,
 1212874910739116032,
 1212874901998178304,
 1212874895497019394,
 1212874890874884096,
 1212874883123728385,
 1212874876295532544,
 1212874875620249601,
 1212874872122171392,
 1212874870733901825,
 1212874841772220423,
 1212874840216084480,
 1212874834096599048,
 1212874829596184577,
 1212874828572704769]

In [19]:
# this works for search result of year 2020 but not for 2011
# is full text only available for recent years?

status = api.get_status(1212874890874884096, tweet_mode="extended")
try:
    print(status.retweeted_status.full_text)
except AttributeError:  # Not a Retweet
    print(status.full_text)

Happy New Year 2020! For a #healthy start, why not try TIANA 100% Pure MCT Energy, a natural, #vegan, pure and powerful supplement to beat tiredness and boost metabolism, easy to mix in your #morning #coffee, #tea, #smoothies and much more. #energy #bodyenergy #veganuary #mctoil https://t.co/IelmqqYvHW


In [26]:
# myfile.write(json.dumps(status._json))

with open('data.json', 'w') as fp:
    json.dump(status._json, fp)

In [28]:
raw=[]
raw.append(status._json)

raw.append(status._json)

In [35]:
raw

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Thu Jan 02 23:14:42 +0000 2020',
  'display_text_range': [0, 140],
  'entities': {'hashtags': [{'indices': [45, 53], 'text': 'healthy'},
    {'indices': [112, 118], 'text': 'vegan'}],
   'symbols': [],
   'urls': [],
   'user_mentions': [{'id': 769940108,
     'id_str': '769940108',
     'indices': [3, 16],
     'name': 'TIANA Fairtrade Organics',
     'screen_name': 'TIANAcoconut'}]},
  'favorite_count': 0,
  'favorited': False,
  'full_text': 'RT @TIANAcoconut: Happy New Year 2020! For a #healthy start, why not try TIANA 100% Pure MCT Energy, a natural, #vegan, pure and powerful s…',
  'geo': None,
  'id': 1212874890874884096,
  'id_str': '1212874890874884096',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'place': None,
  'retweet_count': 2,
  'retweet

In [18]:
# search a list of tweet ids
status2=api.statuses_lookup(idlist,tweet_mode="extended")
for status in status2:
    try:
        print(status.retweeted_status.full_text)
    except AttributeError:  # Not a Retweet
        print(status.full_text)


NameError: name 'idlist' is not defined

In [7]:
# basic timeline for Anran's account
public_tweets = api.home_timeline()
for tweet in public_tweets:     
    print(tweet.text, tweet.created_at) 

RT @UoDLibrary: Library in your Living Room - Advanced Endnote: backing up your files  Thursday 29 April @ 12.30 Online

Confused when tryi… 2021-04-28 09:31:40
RT @DundeeDA: There is still time to apply for our 2021 Three Minute Thesis competition!
Submit your abstract here 👉https://t.co/NjL8bsmz2O… 2021-04-28 09:30:18
RT @turnermaround: https://t.co/iiLvOcWBpM 2021-04-28 09:30:13
RT @turnermaround: Very honoured to have been included in “Agile an Unexpected Journey” and in very good company. 

Available through Amazo… 2021-04-28 09:29:55
RT @UoDCulture: As it was recently World Malaria day, we would like to draw attention to a talk by Dr Beatriz Baragana and Irene Halliburto… 2021-04-28 09:27:43
I used to consider myself a good writer, one who could play with words. I had done my homework and had a sophistica… https://t.co/OLpEbfHyli 2021-04-28 09:24:23
Congratulations @lanky_manky and thank you for the acknowledgement!  If you would like to acknowledge the input of… https://t.co/rCq

In [16]:
# Get the User object 
user = api.get_user('AnranZhao')
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
   print(friend.screen_name)

AnranZhao
17
UoM_ITS
BeveragesHot
SorchaUKDS
GillUKDS
CMI_Man
Patty_NZ
VanessaAHiggins
ABloom_20
UKDSImpact
mikybartoli
JKasmireComplex
_JAStark
PyDataMCR
JosephAllen1234
opendatamcr
UKDataService
stephen_whitemn
McrContemporary
OfficialUoM
arthistsoc_uom


In [41]:
# collect 1000 English tweets on 'vegan' and show the full text
# but all retweets are not full 
# geolocation restriction is added
for status in tweepy.Cursor(api.search, q='vegan', lang='en', tweet_mode='extended', geocode="31.578871,74.305184,5km" ).items(1000):
    print (status.full_text)

Vegan approach towards life is not about growing another kind of food industry though people are free to eat and invent foods as much they want. It is simple to understand and simpler to adopt this not-cruel lifestyle. https://t.co/SA2lOlr0wi
@vemi_d @MooMeagan @YouDont48355285 @peta Imagine arguing with a vegan😂 https://t.co/lsHQh7192j
@Midtendo @MooMeagan @YouDont48355285 @peta She's a vegan, what else do you expect from her?


In [18]:
# collect 200 tweets from user TheVeganRD and show the full text
# NB, here retweets lost the identification 'RT' and become common tweets

results = api.user_timeline(id='TheVeganRD', count=200, tweet_mode='extended')
for status in results:
    try:
        print(status.retweeted_status.full_text)
    except AttributeError:  # Not a Retweet
        print(status.full_text)

@Mjwenckus Some combination of salt, fat, flavor, texture, umami. After all, the early nondairy cheeses contained casein and they were anything but addictive!
I've been writing vegan advocacy material since the early 1990s. I'm totally disgusted by the lack of credibility in some of the most oft-repeated claims, just as I know my friend @TheVeganRD has equal despair over bogus vegan health claims. 5/
If someone says you can lose weight without being on a "diet" by eating plant-based and then they say you can never have oil or sugar or veggie meats or refined pasta--guess what? They’re putting you on a diet.
@SuchFun_AreWe Agree -- I am a very big fan of Violife! It's on sale right now and my refrigerator is packed with it.
@mdryja I just looked it up -- and now it's on my list for the next time I'm in southern CA.
@tempehlover @moozoogabba @SuchFun_AreWe That cheese looks amazing!
@NCanimalvoters @jan_dutkiewicz Oh, those corn dogs!
Such an interesting article on the history and evolut

In [19]:
# comparison
results = api.user_timeline(id='TheVeganRD', count=200, tweet_mode='extended')
for status in results:
    print(status.full_text)

@Mjwenckus Some combination of salt, fat, flavor, texture, umami. After all, the early nondairy cheeses contained casein and they were anything but addictive!
RT @vegan: I've been writing vegan advocacy material since the early 1990s. I'm totally disgusted by the lack of credibility in some of the…
If someone says you can lose weight without being on a "diet" by eating plant-based and then they say you can never have oil or sugar or veggie meats or refined pasta--guess what? They’re putting you on a diet.
@SuchFun_AreWe Agree -- I am a very big fan of Violife! It's on sale right now and my refrigerator is packed with it.
@mdryja I just looked it up -- and now it's on my list for the next time I'm in southern CA.
@tempehlover @moozoogabba @SuchFun_AreWe That cheese looks amazing!
@NCanimalvoters @jan_dutkiewicz Oh, those corn dogs!
Such an interesting article on the history and evolution of #vegan cheese. (Just skip over the questionable claims about "addictive" properties of casein.)  

In [22]:
# test: without cursor, only tweets on the first result page is shown, which is 100 tweets here
count=0
res=api.search(q='veganism', lang='en',tweet_mode='extended', count=100000)
for tweet in res:
    count+=1
    print(tweet.full_text)
    print(count)

Veganism is a social justice movement. The way we fight for slavery, racism, women rights... now is time to fight for animals. #GoVegan #endspeciesism
1
Veganism &amp; spirituality do not go hand and hand. Argue with yo mama while I eat my cheese &amp; chicken nuggies. https://t.co/BDM4W4y7tF
2
@jewish_activist the way most leftists talk about animal rights is on par with how ben shapiro “debates”. for all the complaints about “white veganism” they sure love using BIPOC people as shields for their own shitty habits
3
RT @i_am_RAGHAV: There is no argument against veganism. Stop being a plant rights activist. Stop the slaughter, exploitation, enslavement,…
4
@makeyallsweat @Klanderverdig @milkthecashews Firstly, I have clarified several times, vegans have contracted the virus. Secondly, I stand by my choice and the right to choose without being judged. Thirdly, you say veganism can't fix it, when the science says Covid-19 jumped from animals to humans. Same as HIV, SARS etc.
5
RT @Veggie

In [33]:
# store only text and time

df=pd.DataFrame()
content=[]
time=[]

search=api.search(q='COVID', lang='en', count=5,tweet_mode='extended')
for tweet in search:
    content.append(tweet.full_text)
    time.append(tweet.created_at )
    
df['content']=content
df['time']=time
df

,content,time
0,RT @PChidambaram_IN: The Centre is targeting M...,2021-04-08 13:55:22
1,RT @ShapedInternet: It's been a year since we ...,2021-04-08 13:55:22
2,RT @ArshiyaMuskaan7: Respected @DrTamilisaiGuv...,2021-04-08 13:55:22
3,RT @WebMD: The Pfizer and Moderna COVID-19 vac...,2021-04-08 13:55:21
4,RT @PoliticsForAlI: I would be more likely to ...,2021-04-08 13:55:21


In [40]:
# store all features
# ref: from 26min, https://www.youtube.com/watch?v=dvAurfBB6Jk&t=2637s&ab_channel=CodingEntrepreneurs 
# need further editing. what's difference between author and user?


search=api.search(q='vegan', lang='en', count=300,tweet_mode='extended')

columns = set()
allowed_types = [str, int]
tweets_data = []
for status in search:
    status_dict = dict(vars(status))
    keys = status_dict.keys()
    single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
    for k in keys:
        try:
            v_type = type(status_dict[k])
        except:
            v_type = None
        if v_type != None:
            if v_type in allowed_types:
                single_tweet_data[k] = status_dict[k]
                columns.add(k)
    tweets_data.append(single_tweet_data)

        
header_cols = list(columns)
header_cols.append("user")
header_cols.append('author')

df = pd.DataFrame(tweets_data, columns=header_cols)
df

,retweet_count,in_reply_to_status_id,source,id_str,in_reply_to_status_id_str,quoted_status_id,id,full_text,source_url,in_reply_to_user_id_str,quoted_status_id_str,favorite_count,in_reply_to_user_id,in_reply_to_screen_name,lang,user,author
0,5,NaN,Twitter for iPhone,1387348882942795776,NaN,NaN,1387348882942795776,RT @Centr_Ale: A welcome returner from @Anarch...,http://twitter.com/download/iphone,NaN,NaN,0,NaN,NaN,en,beerpornstar,beerpornstar
1,0,NaN,Instagram,1387348834616070147,NaN,NaN,1387348834616070147,Pamper your mum on Mothers day. FREE DELIVERY ...,http://instagram.com,NaN,NaN,0,NaN,NaN,en,DivinityCollect,DivinityCollect
2,0,NaN,RecipeTweeter,1387348720207945729,NaN,NaN,1387348720207945729,Smoky Stuffed Cabbage Leaves with Miso Tahini ...,http://inseasonrecipes.co.uk/,NaN,NaN,0,NaN,NaN,en,InSeasonRecipes,InSeasonRecipes
3,0,1.386655e+18,Twitter for Android,1387348702478614533,1386655022323322883,NaN,1387348702478614533,@carav4ggio Vegan energy drink,http://twitter.com/download/android,1057257167017918469,NaN,0,1.057257e+18,carav4ggio,en,Ashleydmackle,Ashleydmackle
4,0,1.386748e+18,Twitter Web App,1387348691174891521,1386747538137550848,NaN,1387348691174891521,@MrBeast @MrBeast Can you make some burgers ve...,https://mobile.twitter.com,2455740283,NaN,0,2.455740e+09,MrBeast,en,premiumpizza1,premiumpizza1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,NaN,Stamped.io,1387346412837343233,NaN,NaN,1387346412837343233,⭐⭐⭐⭐⭐ 5 star review from Deanna C. https://t.c...,https://www.stamped.io,NaN,NaN,0,NaN,NaN,en,CandleShackLtd,CandleShackLtd
96,0,1.387113e+18,Twitter for Android,1387346404889202688,1387113431279538180,NaN,1387346404889202688,@helenem06 @BrexitBassist So are you a vegan?,http://twitter.com/download/android,20232580,NaN,0,2.023258e+07,helenem06,en,ATestingWorld,ATestingWorld
97,2171,NaN,Twitter for Android,1387346401814851586,NaN,NaN,1387346401814851586,RT @ladygaga: Kiss harder and SMARTER with PhD...,http://twitter.com/download/android,NaN,NaN,0,NaN,NaN,en,lmy_emma,lmy_emma
98,0,1.387330e+18,Twitter for Android,1387346379056451585,1387329530361225217,NaN,1387346379056451585,@GregCampNC @ThunderingOx @FlintyMaguire @Fras...,http://twitter.com/download/android,496199344,NaN,0,4.961993e+08,GregCampNC,en,AuntieJudee,AuntieJudee


In [33]:
df.loc[0].content

'RT @MSNBC: WATCH: President Biden urges Americans to continue following Covid-19 guidelines: "Mask up, it’s a patriotic duty. It’s the only…'